<a href="https://colab.research.google.com/github/Andre6o6/stylegan-editing/blob/master/StyleGAN_edit_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook shows the example of image editing

In [ ]:
#@title Load dependences {display-mode: "form"}

%tensorflow_version 1.x

!git clone https://github.com/Andre6o6/stylegan-editing.git stylegan_editing
%cd stylegan_editing

#TODO as submodule
!git clone https://github.com/genforce/interfacegan.git

# Load pretrained models
!gdown --id 1r3Qygz6DaXtQwkUbd35ucA2U4hayj32m
!mv karras2019stylegan-ffhq-1024x1024.pkl interfacegan/models/pretrain/
!gdown --id 1C9MSghPDWnkccGXgU6S9-wnRgPVBVovL

!pip install facenet-pytorch

!pip install ffmpeg-python
!pip install scikit-video


---

In [2]:
#@title Upload your images {display-mode: "form"}

import os
from google.colab import files

os.makedirs("latents", exist_ok=True)
os.makedirs("raw_images", exist_ok=True)
os.makedirs("aligned_images", exist_ok=True)

uploaded = files.upload()
for fn in uploaded.keys():
    os.rename(fn, os.path.join("raw_images", fn))

In [ ]:
#@title (Optional) or download images, picked from FEI Face Database {display-mode: "form"}

!mkdir latents/ raw_images/ aligned_images/

%cd raw_images/
!gdown --id 1qnf8pbtTmBguMUAre0ZeC1F_DqpWnFcp
#!gdown --id 1eFKVKEGsGr9Yl-uI3lC3XaqthAX8uPl4
!gdown --id 1KzXBdEvTyhyyWuPwJpGC05Ix6HU1foVm
#!gdown --id 1vPOidmfgxvsks3hfqOya_WfNElkwlcDo
#!gdown --id 1hkQZH8DoPioUMkr60S5MM3OjYlJgvnK5
!gdown --id 1Br3jI1ae2T0eN0TNHDfistlb1fshLQdR
%cd ..


---

In [ ]:
#@title Align images {display-mode: "form"}

!python align/align_images.py raw_images/ aligned_images/ --output_size=1024


---

In [ ]:
#@title Encode images in StyleGAN latent space... {display-mode: "form"}
#@markdown This will take some time.
#@markdown We set number of iterations to 400 to get decent quality on real images, so it takes around 3 min per image. 

!python encode_images.py

In [ ]:
#@title ...and show results: {display-mode: "form"}

import numpy as np
import os
from PIL import Image

import torch
import torch.nn as nn
import torch.functional as F

from interfacegan.models.stylegan_generator import StyleGANGenerator
from models.latent_optimization import LatentOptimizer
from utils.io import get_image_numpy

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
converted_model = StyleGANGenerator("stylegan_ffhq")
latent_optimizer = LatentOptimizer(converted_model.model)

print("\n\t\tInput\t\tReconstruction")
for imgname in os.listdir("aligned_images"):
    img_path = "aligned_images/" + imgname
    latent_path = "latents/" + imgname + ".npy"

    img = np.array(Image.open(img_path))

    latent = np.load(latent_path)
    _, generated_image = latent_optimizer(torch.from_numpy(latent).to(device))
    g = get_image_numpy(generated_image)

    imgs_comb = np.hstack((img, g))
    display(Image.fromarray(imgs_comb).resize((512,256)))


---

## Edit image

In [ ]:
#@title Run UI

import numpy as np
import os
from PIL import Image
import ipywidgets as widgets

#Dropdown lists to choose images
img_list = os.listdir("aligned_images/")
input_picker = widgets.Dropdown(options=img_list)
exemplar_picker = widgets.Dropdown(options=img_list)
feature_picker = widgets.Dropdown(options=["pose", "smile"], value="pose")

print("Choose exemplar image, from which feature will be transfered:")
display(exemplar_picker)
print("Choose your target image for feature transfer:")
display(input_picker)
print("Choose feature to transfer:")
display(feature_picker)

#Interactive display that shows chosen images
def display_feature_ui(exemplar, input, feature):
    print(f"{feature} transfer: {exemplar} --> {input}")
    paths = ["aligned_images/"+x for x in [exemplar, input]]
    imgs = [Image.open(x).resize((256,256)) for x in paths]
    imgs_comb = np.hstack( [np.asarray(x) for x in imgs] )
    display(Image.fromarray(imgs_comb))

out = widgets.interactive_output(
    display_feature_ui, 
    {"exemplar": exemplar_picker, "input": input_picker, "feature": feature_picker}
)

display(out)

#Strength slider
print("Adjust edit strength:")
scale = widgets.FloatSlider(min=0, max=5, step=0.05, value=1, continuous_update=False)
display(scale)
print("Adjust identity preservation strength:")
print("(Bigger values tend to produse better results, but lessen feature transfer strength.)")
identity = widgets.FloatSlider(min=1, max=10, step=0.5, value=5, continuous_update=False)
display(identity)

In [ ]:
#@title Perform feature transfer... {display-mode: "form"}
#@markdown This will take some time.

import numpy as np
import os
from PIL import Image
from IPython import display as ipythondisplay
from IPython.display import display

import torch
import torch.nn as nn
import torch.nn.functional as F

from facenet_pytorch import InceptionResnetV1
from interfacegan.models.stylegan_generator import StyleGANGenerator
from models.latent_optimization import LatentOptimizer
from edit_images import morph_coefficient, feature_morph
from utils.io import get_image_numpy

# Load models
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
converted_model = StyleGANGenerator("stylegan_ffhq")
latent_optimizer = LatentOptimizer(converted_model.model, latent_space="WP")
facenet = InceptionResnetV1(pretrained='vggface2').to(device).eval()

# Get feature vector
if (feature_picker.value == "smile"):
    boundary = np.load("boundaries/stylegan_ffhq_smile_w_boundary.npy")
elif (feature_picker.value == "pose"):
    boundary = np.load("boundaries/stylegan_ffhq_pose_w_boundary.npy")

# Get latent vectors
w_input = np.load("latents/" + input_picker.value + ".npy")
w_exemplar = np.load("latents/" + exemplar_picker.value + ".npy")
# Calculate edit distance
effect_coef = 2 * scale.value * morph_coefficient(w_input, w_exemplar, boundary, map_k=5)

# Show initial prediction, obtained with just linear shift
print("\nInitial rough feature transfer prediction (not final result):")
temp = (w_input + effect_coef*boundary).astype(np.float32)
_, generated_image = latent_optimizer(torch.from_numpy(temp).to(device))
g = get_image_numpy(generated_image)
display(Image.fromarray(g).resize((256,256)))

# Perform edit
print("Performing feature transfer...")
res, path = feature_morph(w_input, boundary, effect_coef, latent_optimizer, facenet, \
                          identity_coef=identity.value)

ipythondisplay.clear_output(wait=True)
print("Finished.")

# Show final result image
_, generated_image = latent_optimizer(torch.from_numpy(res).to(device))
g = get_image_numpy(generated_image)
display(Image.fromarray(g).resize((512,512)))

In [ ]:
#@title (Optional) Show video {display-mode: "form"}


from skimage.transform import resize
import skvideo.io
from moviepy.editor import VideoFileClip

def gen(x):
    _, generated_image = latent_optimizer(torch.from_numpy(x).to(device))
    g = get_image_numpy(generated_image, saturate=True)
    return resize(g,(512,512))

frames = [gen(x) for x in path]
frames = (255*np.array(frames)).astype(np.uint8)
skvideo.io.vwrite("temp_out.mp4", frames)

clip = VideoFileClip("temp_out.mp4")
clip.ipython_display(height=512, autoplay=1, loop=1)